In [9]:
%%writefile practice72.cu

// Practice 7 Task 2: Префиксная сумма (сканирование) на GPU
// 1. Реализовать алгоритм префиксной суммы (prefix sum) на GPU
// 2. Использовать разделяемую память для ускорения
// 3. Проверить корректность на тестовых массивах

#include <iostream>               // Для стандартного ввода-вывода
#include <cuda_runtime.h>         // Для CUDA Runtime API
#include <chrono>                 // Для измерения времени

using namespace std;              // Чтобы не писать std::

// CUDA KERNEL: ПРЕФИКСНАЯ СУММА В БЛОКЕ
__global__ void prefixSumKernel(int *d_input, int *d_output, int n) {
    extern __shared__ int temp[];                // Разделяемая память блока (shared memory)
    int tid = threadIdx.x;                       // Локальный индекс потока внутри блока
    int i = blockIdx.x * blockDim.x + tid;      // Глобальный индекс элемента массива

    // Загружаем данные из глобальной памяти в shared memory
    temp[tid] = (i < n) ? d_input[i] : 0;       // Если поток не выходит за границы массива
    __syncthreads();                            // Синхронизация потоков блока перед обработкой

    // Шаг вверх (upsweep)
    for (int offset = 1; offset < blockDim.x; offset *= 2) {
        int index = (tid + 1) * offset * 2 - 1; // Индекс элемента для сложения
        if (index < blockDim.x)
            temp[index] += temp[index - offset]; // Складываем элементы
        __syncthreads();                         // Синхронизация после каждой итерации
    }

    // Шаг вниз (downsweep)
    if (tid == 0)
        temp[blockDim.x - 1] = 0;                // Обнуляем последний элемент перед downsweep
    __syncthreads();                              // Синхронизация всех потоков

    for (int offset = blockDim.x / 2; offset > 0; offset /= 2) {
        int index = (tid + 1) * offset * 2 - 1;  // Индекс элемента для распределения суммы
        if (index < blockDim.x) {
            int t = temp[index - offset];        // Сохраняем временное значение
            temp[index - offset] = temp[index];  // Перемещаем значение в предыдущую позицию
            temp[index] += t;                    // Добавляем сохранённое значение
        }
        __syncthreads();                         // Синхронизация перед следующей итерацией
    }

    // Записываем результат обратно в глобальную память
    if (i < n)
        d_output[i] = temp[tid];
}

// ФУНКЦИЯ ЗАПУСКА ОДНОГО ТЕСТА
void runPrefixSumTest(int N) {
    int SIZE = N * sizeof(int);                  // Размер массива в байтах
    int *h_array = new int[N];                   // Создаем массив на CPU
    int *h_result_cpu = new int[N];              // Массив для результата CPU
    int *h_result_gpu = new int[N];              // Массив для результата GPU

    // Заполняем массив единицами
    for (int i = 0; i < N; i++)
        h_array[i] = 1;

    // CPU Префиксная сумма
    auto cpu_start = chrono::high_resolution_clock::now(); // Запуск таймера CPU
    h_result_cpu[0] = h_array[0];                           // Первый элемент равен самому себе
    for (int i = 1; i < N; i++)
        h_result_cpu[i] = h_result_cpu[i - 1] + h_array[i]; // Каждый следующий элемент = сумма предыдущих
    auto cpu_end = chrono::high_resolution_clock::now();   // Остановка таймера CPU
    chrono::duration<double> cpu_time = cpu_end - cpu_start; // Вычисляем время выполнения CPU

    // GPU Префиксная сумма
    int *d_input, *d_output;                              // Указатели на память GPU
    cudaMalloc(&d_input, SIZE);                            // Выделяем память на GPU под входной массив
    cudaMalloc(&d_output, SIZE);                           // Выделяем память на GPU под результат
    cudaMemcpy(d_input, h_array, SIZE, cudaMemcpyHostToDevice); // Копируем данные CPU -> GPU

    int threadsPerBlock = 1024;                             // Потоки в одном блоке
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock; // Количество блоков

    cudaEvent_t gpu_start, gpu_stop;                       // Объявляем события для таймера GPU
    cudaEventCreate(&gpu_start);                           // Создаем событие старта GPU
    cudaEventCreate(&gpu_stop);                            // Создаем событие конца GPU
    cudaEventRecord(gpu_start);                            // Запуск таймера GPU

    prefixSumKernel<<<blocksPerGrid, threadsPerBlock, threadsPerBlock * sizeof(int)>>>(d_input, d_output, N); // Запуск ядра GPU

    cudaEventRecord(gpu_stop);                             // Остановка таймера GPU
    cudaEventSynchronize(gpu_stop);                        // Синхронизация с GPU

    float gpu_time = 0;
    cudaEventElapsedTime(&gpu_time, gpu_start, gpu_stop);   // Получаем время выполнения GPU

    cudaMemcpy(h_result_gpu, d_output, SIZE, cudaMemcpyDeviceToHost); // Копируем результат GPU -> CPU

    // Вывод только времени
    cout << "\n Размер массива: " << N << endl;            // Вывод размера массива
    cout << "CPU время: " << cpu_time.count() * 1000 << " мс" << endl; // Вывод времени CPU
    cout << "GPU время: " << gpu_time << " мс" << endl;   // Вывод времени GPU

    // Освобождение памяти
    delete[] h_array;                                      // Освобождаем память CPU
    delete[] h_result_cpu;                                 // Освобождаем память CPU
    delete[] h_result_gpu;                                 // Освобождаем память CPU
    cudaFree(d_input);                                     // Освобождаем память GPU
    cudaFree(d_output);                                    // Освобождаем память GPU
}

// Основная функция
int main() {
    runPrefixSumTest(1024);        // Тест для массива из 1024 элементов
    runPrefixSumTest(1000000);     // Тест для массива из 1 000 000 элементов
    runPrefixSumTest(10000000);    // Тест для массива из 10 000 000 элементов
    return 0;                      // Завершаем программу
}


Overwriting practice72.cu


In [10]:
# Компиляция
!nvcc practice72.cu -o practice72 -arch=sm_75 -std=c++11            # -arch=sm_75  - архитектура GPU (Tesla T4 в Colab = sm_75)
                                                                    # -std=c++11 — стандарт C++
# Запуск
!./practice72


 Размер массива: 1024
CPU время: 0.003083 мс
GPU время: 0.1416 мс

 Размер массива: 1000000
CPU время: 4.7016 мс
GPU время: 0.413888 мс

 Размер массива: 10000000
CPU время: 44.8747 мс
GPU время: 3.98112 мс
